In [8]:
import numpy as np
import matplotlib.pyplot as plt
import gmsh
import os
import meshio
import math
import sys
from matplotlib.patches import Polygon
from matplotlib.tri import Triangulation
import matplotlib.cm as cm  # Importar módulo de colormap
from CST import CST
from Node import Node
from units import mm, cm, m, kgf, N, tf, kN, MPa, GPa
from fem import Material, Membrane
from functions import matrix_extract,  get_nodes_from_physical_id, get_line_load_global_vector, matrix_replace

In [9]:
output_file = "mallas/malla_CST.msh"   #cambiar el numero de la malla --> malla_quad4G1.msh, malla_quad4G2.msh, malla_quad4G3.msh, malla_quad4G4.msh, malla_quad4L1.msh,....
geo_file= "T4.geo"
titulo = "Quad 4 Local refinement case 4"#"Quad 9 Global refinement case 1" #Cambiar el numero y G-->L

In [10]:
section = {"Omega"}

In [11]:
def load_mesh_objects(geo_file=geo_file, msh_file=output_file):

    # Leer malla
    mesh = meshio.read(msh_file)

    # Crear nodos
    nodes = [Node(i+1, x, y) for i, (x, y, _) in enumerate(mesh.points)]

    # Crear elementos CST
    cst_elements = []
    for cell_block in mesh.cells:
        if cell_block.type == "triangle":
            for i, node_ids in enumerate(cell_block.data):
                node_ids = [int(id) + 1 for id in node_ids]  # Convertir a índices base 1
                
                cst_elements.append(CST(i+1 , list(node_ids)))
            break

    # Detectar nodos en líneas físicas ("Diritchlet 1" a "Diritchlet 4")
    boundary_nodes = {1: set(), 2: set(), 3: set(), 4: set()}

    for i, cell_block in enumerate(mesh.cells):
        if cell_block.type == "line":
            physical_ids = mesh.cell_data_dict['gmsh:physical']['line']
            for line, phys_id in zip(cell_block.data, physical_ids):
                if phys_id in boundary_nodes:
                    for node_id in line:
                        boundary_nodes[phys_id].add(int(node_id))

    # Añadir etiquetas de borde a los nodos
    for node in nodes:
        node.boundary_label = []
        for label_id, node_set in boundary_nodes.items():
            if node.id in node_set:
                node.boundary_label.append(f"Diritchlet Boundary")

    return nodes, cst_elements


def plot_mesh(nodes, elements, show_node_ids=False, show_elem_ids=False):
    x = np.array([node.x for node in nodes])
    y = np.array([node.y for node in nodes])

    # Convertir elementos CST a array de conectividades
    triangles = np.array([[nid - 1 for nid in elem.node_ids] for elem in elements])


    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_aspect('equal')

    triang = Triangulation(x, y, triangles)
    ax.triplot(triang, color='gray', linewidth=0.8)

    if show_node_ids:
        for node in nodes:
            ax.text(node.x, node.y, str(node.id), color='blue', fontsize=8, ha='center', va='center')

    if show_elem_ids:
        for elem in elements:
            coords = np.array([[nodes[j - 1].x, nodes[j - 1].y] for j in elem.node_ids])

            centroid = coords.mean(axis=0)
            ax.text(*centroid, str(elem.id), color='red', fontsize=8, ha='center')

    ax.set_title("Malla de elementos triangulares (CST)")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.tight_layout()
    plt.show()

In [12]:
nodes, elements = load_mesh_objects(geo_file=geo_file, msh_file=output_file)
plot_mesh(nodes, elements, show_node_ids=True, show_elem_ids=True)

AttributeError: 'int' object has no attribute 'coordenadas'